In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import random

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt             
import matplotlib as mpl                    
mpl.rc('font', family='Malgun Gothic')      
plt.rcParams['axes.unicode_minus'] = False  

import tensorflow as tf
from tensorflow import keras  
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam
from keras.models import load_model

from sklearn import metrics

In [3]:
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# DCGAN
- Deep Convolutional Generative Adversarial Networks 
- 데이터 전처리 >> 생성자 모델 >> 판매자 모델 >> 컴파일 >> 학습 >> 예측
- https://www.tensorflow.org/tutorials/generative/dcgan?hl=ko

## 생성자 모델 (Generator Model)

In [22]:
# 생성자 모델 구성
g_model=keras.Sequential()
g_model.add(keras.layers.Input(shape=(100, )))               

# Generator 내부에서 노이즈를 128*7*7=6272 1차원
g_model.add(keras.layers.Dense(128*7*7))       
g_model.add(keras.layers.LeakyReLU(0.2))        # 0보다 작을 경우 0.2 곱하기   
g_model.add(keras.layers.BatchNormalization())               

# 7×7 → 14×14 → 28×28 점점 키워간다. 
g_model.add(keras.layers.Reshape((7, 7, 128)))  #  CNN 레이어가 받아들일 수 있는 형태 7 7 128

g_model.add(keras.layers.UpSampling2D())         # 14 14 128:  두 배씩 업(up)샘플링
g_model.add(keras.layers.Conv2D(64, kernel_size=5, padding='same'))   # 14 14 64 Conv2DTranspose() 사용가능 
g_model.add(keras.layers.BatchNormalization())    
g_model.add(keras.layers.LeakyReLU(0.2))
 
g_model.add(keras.layers.UpSampling2D())    # 28 28 64 : 두 배씩 업(up)샘플링
g_model.add(keras.layers.Conv2D(1, kernel_size=5, padding='same', activation='tanh'))  # 28 28 1
# g_model.summary()

## 판별자 모델 (Discriminator Model)
- 가짜인지, 진짜 인지만 판별 >> 자기 자신은 학습을 하면 안됨
- 판별자가 얻은 가중치는 판별자 자신이 학습하는 데 쓰이는 것이 아니라 <br>
  생성자로 넘겨주어 생성자가 업데이트된 이미지를 만들도록 해야 함

In [25]:
d_model=keras.Sequential()
d_model.add(keras.layers.Input(shape=(28, 28, 1)))

# MaxPooling은 정보를 너무 과감하게 버릴 수 있음
# strides 다운샘플링과 특징 추출을 한 번에 처리할 수 있어서 연산 효율이 높고 간단한 구조 유지 
d_model.add(keras.layers.Conv2D(64, kernel_size=5, strides=2, padding='same'))   # 14 14 64
d_model.add(keras.layers.LeakyReLU(0.2))
d_model.add(keras.layers.Dropout(0.3))    # 14 14 64

d_model.add(keras.layers.Conv2D(128, kernel_size=5, strides=2, padding='same'))  # 7 7 128
d_model.add(keras.layers.LeakyReLU(0.2))
d_model.add(keras.layers.Dropout(0.3))   # 7 7 128

d_model.add(keras.layers.Flatten())   # 7*7*128=6272 
d_model.add(keras.layers.Dense(1, activation='sigmoid'))   # 6,273 
# d_model.summary()

# 논문 DCGAN, 2015 기준
# 학습률 : 학습이 천천히 진핼되도록 >> 작은 값이 안정성에 도움
# 모멘텀 : 이전 단계의 그래디언트를 일정 비율만큼 유지 (이전 방향을 50% 반영)
d_model.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy']) 
d_model.trainable=False                # 판별이 끝나고 나면 판별자 자신이 학습되지 않게끔 학습 기능을 꺼줌

## GAN 모델 생성 - 판별자 + 생성자 연결 모델 구성

In [28]:
g_input=keras.Input(shape=(100,))      # 100, 랜덤한 100개의 벡터
d_output=d_model(g_model(g_input))     # 노이즈 입력 >> 28, 28, 1 결과 >> 참, 거짓

gan=Model(g_input, d_output)   
gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])    # 참과 거짓 구분
gan.summary()

Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_2 (Sequential)            │ (None, 28, 28, 1)           │         865,281 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_3 (Sequential)            │ (None, 1)                   │         212,865 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,078,146 (4.11 MB)

 Trainable params: 852,609 (3.25 MB)

 Non-trainable params: 225,537 (881.00 KB)

## 학습 함수

In [31]:
def get_train(epoch, batch_size, saving_interval):
    (X_train, _), (_, _)=keras.datasets.mnist.load_data()
    X_train=(X_train.reshape(-1, 28, 28, 1)  / 255.0) * 2 - 1   # tanh 함수로 정규화 -1 ~ 1 
    
    true=np.ones((batch_size, 1))     # 1 레이블
    fake=np.zeros((batch_size, 1))    # 0 레이블

    history=[]
    for i in range(epoch):
        # 1) 실제 데이터를 판별자에 입력
        d_model.trainable=True
        idx=np.random.randint(0, X_train.shape[0], batch_size)    # 0 ~ 60000 사이의 숫자 중 랜덤하게 32번 반복
        imgs=X_train[idx]  # 선택한 이미지 가져옴
        d_loss_real=d_model.train_on_batch(imgs, true)   # 한 번의 배치만 수동으로 학습

        # 2) 가상 이미지를 판별자에 입력
        noise=np.random.normal(0, 1, (batch_size, 100))   # 실수 0 ~ 1 사이의 숫자 중 랜덤하게 32번 반복하는데 100열 뽑아라
        g_imgs=g_model.predict(noise, verbose=0)
        d_loss_fake=d_model.train_on_batch(g_imgs, fake) 

        # ---------------------------------------
        # 이제 실제 이미지를 넣은 d_loss_real과 가상 이미지를 입력한 d_loss_fake가 
        # 판별자 안에서 번갈아 가며 진위를 판단하기 시작  ---------------- #

        # 3) 오차 계산 : (d_loss_real+d_loss_fake) / 2  >> 속도 향상을 위해서 
        d_loss=0.5 * np.add(d_loss_real, d_loss_fake)  # 판별자와 생성자의 오차를 계산
        gan_loss=gan.train_on_batch(noise, true)       # gan모델을 통해서 생성자의 오차 / 무조건 정답이라고 하고 넘김

        # 4) 출력
        # print('epoch:%d' % i, ' d_loss:%.4f' % d_loss[0], 'd_accuracy:%.4f' % d_loss[1], 
        #       ' gan_loss:%.4f' % gan_loss[0], 'gen_accuracy:%.4f' % gan_loss[1])

        # 5) 그래프 및 저장
        if i % saving_interval == 0:
            print('epoch:%d' % i, ' d_loss:%.4f' % d_loss[0], 'd_accuracy:%.4f' % d_loss[1], 
                  ' gan_loss:%.4f' % gan_loss[0], 'gen_accuracy:%.4f' % gan_loss[1])
            
            noise=np.random.normal(0, 1, (25, 100))
            g_imgs=g_model.predict(noise, verbose=0)
            g_imgs=0.5 * g_imgs + 0.5   # tanh >> 0 ~ 255

            fig, axs=plt.subplots(5, 5)
            count=0
            for j in range(5):
                for k in range(5):
                    axs[j, k].imshow(g_imgs[count, :, :, 0], cmap='gray')
                    axs[j, k].axis('off')
                    count +=1
            fig.savefig("./Deep_result/gen_images/gan_mnist_%d.png" % i)
            plt.close(fig)

In [33]:
# 2000번 반복되고(+1을 하는 것에 주의),
# 배치 크기는 32, 200번마다 결과가 저장됩니다.
# get_train(2001, 32, 200)

get_train(2001, 32, 200)    # 5000 ~ 10000

epoch:0  d_loss:0.6731 d_accuracy:0.5703  gan_loss:0.6081 gen_accuracy:0.9688
epoch:200  d_loss:0.5719 d_accuracy:0.5015  gan_loss:0.4684 gen_accuracy:0.7282
epoch:400  d_loss:0.6451 d_accuracy:0.5026  gan_loss:0.7105 gen_accuracy:0.4513
epoch:600  d_loss:0.6995 d_accuracy:0.5031  gan_loss:0.7806 gen_accuracy:0.3679
epoch:800  d_loss:0.7363 d_accuracy:0.5046  gan_loss:0.8280 gen_accuracy:0.3329
epoch:1000  d_loss:0.7583 d_accuracy:0.5086  gan_loss:0.8666 gen_accuracy:0.3158
epoch:1200  d_loss:0.7752 d_accuracy:0.5130  gan_loss:0.9082 gen_accuracy:0.3016
epoch:1400  d_loss:0.7892 d_accuracy:0.5165  gan_loss:0.9486 gen_accuracy:0.2920
epoch:1600  d_loss:0.8034 d_accuracy:0.5194  gan_loss:0.9826 gen_accuracy:0.2856
epoch:1800  d_loss:0.8165 d_accuracy:0.5223  gan_loss:1.0201 gen_accuracy:0.2784
epoch:2000  d_loss:0.8276 d_accuracy:0.5248  gan_loss:1.0500 gen_accuracy:0.2743


In [ ]:
#  X_train=np.expand_dims(X_train, axis=-1) 